In [9]:
# !pip install geemap retry

In [73]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

import ee
import geemap
# Authenticate to Earth Engine.
service_account = 'ee-python@ee-devajji.iam.gserviceaccount.com'
credentials = ee.ServiceAccountCredentials(service_account, './ee-devajji-8565344166d4.json')
ee.Initialize(credentials, project='ee-devajji', opt_url='https://earthengine-highvolume.googleapis.com')

import pandas as pd
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np

import logging
import multiprocessing
import os
import requests
import shutil
import zipfile
import io
from retry import retry

In [32]:
# Defaults
SCALE = 100
PROJECTION = 'EPSG:32616'

# Define a map object
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [33]:
# Here I draw a sqaure around Bloomington
roi = Map.user_roi
roi

ee.Geometry({
  "functionInvocationValue": {
    "functionName": "GeometryConstructors.Polygon",
    "arguments": {
      "coordinates": {
        "constantValue": [
          [
            [
              -86.666273,
              39.107042
            ],
            [
              -86.666273,
              39.22576
            ],
            [
              -86.419091,
              39.22576
            ],
            [
              -86.419091,
              39.107042
            ],
            [
              -86.666273,
              39.107042
            ]
          ]
        ]
      },
      "geodesic": {
        "constantValue": false
      }
    }
  }
})

In [72]:
def mask_clouds(image):
    # Remove cloud pixels from the image
    qa = image.select('QA_PIXEL').bitwiseAnd(int('111111', 2)).eq(0)
    return image.updateMask(qa)

def apply_scale_factors(image):
    # Scale the bands of the image to convert the values to human readable format
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0).subtract(273.15)
    return image.addBands(optical_bands, None, True).addBands(thermal_bands, None, True)

def calculate_indices(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

# Load datasets
# Digital Elevation Model
nasadem = ee.Image("NASA/NASADEM_HGT/001")

# Landsat Collection
landsat_collection = (
    ee.ImageCollection('LANDSAT/LC09/C02/T1_L2')
    .filterDate('2023-01-01', '2024-01-01')
    .filterBounds(roi)
    .map(mask_clouds)
    .map(apply_scale_factors)
    .map(calculate_indices)
    .median()
)

MODEL_BANDS = ['SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B6', 'NDVI', 'ST_B10']

# Add elevation data

landsat_imagery = landsat_collection.addBands(nasadem).select(MODEL_BANDS)

# import the training polygons - each polygon has a band LCZ which defines the classification
# of each polygon.
# Note - training polygons are only for Bloomington and Indianapolis
training_polygons = geemap.geojson_to_ee("training_data.geojson")
training_polygons = training_polygons.filter(ee.Filter.neq("first", 1))
training_polygons = training_polygons.filter(ee.Filter.neq("first", 4))
training_polygons = training_polygons.filter(ee.Filter.neq("first", 7))
training_polygons = training_polygons.filter(ee.Filter.neq("first", 10))
training_polygons = training_polygons.filter(ee.Filter.neq("first", 16))


bloom_training_image = training_polygons.reduceToImage(properties=["LCZ"], reducer=ee.Reducer.first()).rename('LCZ').toInt()

# Get all polygons in the ROI.
bloom_training_polygons = (
    bloom_training_image
    .reduceToVectors(
        reducer=ee.Reducer.countEvery(),
        geometry=roi,
        scale=SCALE,
        eightConnected=False,
        labelProperty="first"
    )
)

training_samples = landsat_imagery.sampleRegions(
    collection=bloom_training_polygons,
    properties=['first'],
    scale=SCALE,
    geometries=True
)

In [76]:
# Function to extract and download a patch for each polygon
def extract_and_download_patch(feature, i):
    # Get the geometry of the feature
    geom = feature.geometry()
    
    # Get the LCZ class
    lcz_class = feature.get('first').getInfo()
    
    # Buffer the geometry slightly to create a patch
    buffered_geom = geom.buffer(60)  # Buffer by 60 meters to create a 120m×120m patch
    
    # Clip the landsat imagery to the buffered geometry
    patch = landsat_imagery.clip(buffered_geom)
    
    # Create a filename based on the LCZ class and index
    filename = f'sat_images/lcz_{lcz_class}_patch_{i}.tif'
    
    # Get the download URL for the patch
    url = patch.getDownloadURL({
        'scale': 30,
        'crs': 'EPSG:4326',
        'fileFormat': 'GeoTIFF',
        'region': buffered_geom
    })
    
    # Download the file
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # If the response is a zip file, extract the TIF
        if response.headers.get('Content-Type') == 'application/zip':
            with zipfile.ZipFile(io.BytesIO(response.content)) as z:
                # Extract the first TIF file found
                tif_files = [f for f in z.namelist() if f.endswith('.tif')]
                if tif_files:
                    with open(filename, 'wb') as f:
                        f.write(z.read(tif_files[0]))
                    print(f"Downloaded patch {i} for LCZ class {lcz_class}")
                    return {'lcz_class': lcz_class, 'filename': filename}
        else:
            # If it's directly a TIF file
            with open(filename, 'wb') as f:
                f.write(response.content)
            print(f"Downloaded patch {i} for LCZ class {lcz_class}")
            return {'lcz_class': lcz_class, 'filename': filename}
    else:
        print(f"Failed to download patch {i} for LCZ class {lcz_class}: {response.status_code}")
        return None

# Get the features as a list
features = bloom_training_polygons.toList(bloom_training_polygons.size())

# Download patches for each polygon (limit to a reasonable number for testing)
max_patches = 100  # Adjust based on your needs
patch_metadata = []

for i in range(min(max_patches, features.size().getInfo())):
    feature = ee.Feature(features.get(i))
    metadata = extract_and_download_patch(feature, i)
    if metadata:
        patch_metadata.append(metadata)

# Save metadata to CSV for later use
pd.DataFrame(patch_metadata).to_csv('lcz_patches/metadata.csv', index=False)


Downloaded patch 0 for LCZ class 9
Downloaded patch 1 for LCZ class 14
Downloaded patch 2 for LCZ class 3
Downloaded patch 3 for LCZ class 6
Downloaded patch 4 for LCZ class 6
Downloaded patch 5 for LCZ class 9
Downloaded patch 6 for LCZ class 11
Downloaded patch 7 for LCZ class 15
Downloaded patch 8 for LCZ class 8
Downloaded patch 9 for LCZ class 8
Downloaded patch 10 for LCZ class 16
Downloaded patch 11 for LCZ class 14
Downloaded patch 12 for LCZ class 11
Downloaded patch 13 for LCZ class 11
Downloaded patch 14 for LCZ class 8
Downloaded patch 15 for LCZ class 6
Downloaded patch 16 for LCZ class 11
Downloaded patch 17 for LCZ class 9
Downloaded patch 18 for LCZ class 14
Downloaded patch 19 for LCZ class 14
Downloaded patch 20 for LCZ class 6
Downloaded patch 21 for LCZ class 6
Downloaded patch 22 for LCZ class 9
Downloaded patch 23 for LCZ class 5
Downloaded patch 24 for LCZ class 9
Downloaded patch 25 for LCZ class 5
Downloaded patch 26 for LCZ class 12
Downloaded patch 27 for LCZ

OSError: Cannot save file into a non-existent directory: 'lcz_patches'

In [71]:
def create_grid_points(roi, stride_meters, buffer_meters):
    """Create a grid of points within the ROI"""
    # Get ROI bounds
    bounds = roi.bounds().getInfo()['coordinates'][0]
    xmin, ymin = bounds[0]
    xmax, ymax = bounds[2]
    
    # Create grid
    grid_points = []
    x = xmin
    while x < xmax:
        y = ymin
        while y < ymax:
            point = ee.Geometry.Point([x, y])
            if roi.contains(point).getInfo():
                grid_points.append(ee.Feature(point))
            y += stride_meters
        x += stride_meters
    
    return ee.FeatureCollection(grid_points)


def extract_patches_from_ee(landsat_imagery, training_polygons, roi, patch_size=64, stride=32, max_patches=10000):
    """
    Extract patches directly from Earth Engine without downloading full rasters
    
    Args:
        landsat_imagery: Earth Engine image with all bands
        training_polygons: EE FeatureCollection with LCZ classes
        roi: Region of interest
        patch_size: Size of patches to extract
        stride: Stride between patches
        max_patches: Maximum number of patches to extract
        
    Returns:
        X: Patch data as numpy array (n_patches, n_bands, patch_size, patch_size)
        y: Labels as numpy array (n_patches)
    """
    # Get the LCZ image from training polygons
    lcz_image = (
        training_polygons.reduceToImage(properties=["LCZ"], reducer=ee.Reducer.first())
        .rename('LCZ')
        .toInt()
    )
    
    # Combine all bands into a single image
    combined_image = landsat_imagery.addBands(lcz_image)
    
    # Create a grid of points for patch centers
    grid = create_grid_points(roi, stride, patch_size//2)
    
    # Sample the image at each grid point
    samples = combined_image.sampleRegions(
        collection=grid,
        scale=30,  # Landsat resolution
        geometries=True
    )
    
    # Get the sample data as a list
    sample_list = samples.toList(samples.size())
    
    # Limit the number of samples if needed
    sample_size = min(sample_list.size().getInfo(), max_patches)
    
    # Initialize lists to store patch data and labels
    X = []
    y = []
    
    # Process each grid point
    for i in range(sample_size):
        point = ee.Feature(sample_list.get(i))
        point_geom = point.geometry()
        
        # Define a square region centered on the point
        square = point_geom.buffer(patch_size * 15, 1).bounds()  # 30m * patch_size/2
        
        # Get patch data
        patch_data = combined_image.clip(square).getRegion(
            region=square,
            scale=30
        ).getInfo()
        
        # Process patch data
        if len(patch_data) > 1:  # Skip if no data
            # Convert to numpy array
            patch_array = process_ee_patch(patch_data, patch_size, len(MODEL_BANDS))
            
            # Get the label (LCZ class)
            lcz_value = point.get('LCZ').getInfo()
            
            # Add to lists
            if patch_array is not None and lcz_value > 0:
                X.append(patch_array)
                y.append(lcz_value)
    
    # Convert lists to numpy arrays
    X = np.array(X)
    y = np.array(y)
    
    return X, y

X, y = extract_patches_from_ee(landsat_imagery, training_polygons, roi, patch_size=64, stride=32, max_patches=10000)

EEException: Collection.toList: The value of 'count' must be positive. Got: 0.

In [65]:
def process_ee_patch(patch_data, patch_size, num_bands):
    """Process patch data from Earth Engine into a numpy array"""
    # Extract headers
    headers = patch_data[0]
    
    # Find indices of band columns
    band_indices = []
    for i in range(4, 4 + num_bands):  # Band data starts at index 4
        band_indices.append(i)
    
    # Find LCZ index
    lcz_index = headers.index('LCZ')
    
    # Extract data rows
    data_rows = patch_data[1:]
    
    # Create empty arrays
    patch_array = np.zeros((num_bands, patch_size, patch_size))
    
    # Fill arrays with data
    for row in data_rows:
        # Get pixel coordinates
        x_rel = int(row[0]) - int(data_rows[0][0])
        y_rel = int(row[1]) - int(data_rows[0][1])
        
        # Skip if outside patch bounds
        if x_rel >= patch_size or y_rel >= patch_size:
            continue
        
        # Fill band data
        for i, band_idx in enumerate(band_indices):
            if row[band_idx] is not None:
                patch_array[i, y_rel, x_rel] = float(row[band_idx])
    
    return patch_array

class LandsatLCZPatchDataset(Dataset):
    """PyTorch Dataset for Landsat-LCZ patches"""
    def __init__(self, X, y, transform=None):
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()
        self.transform = transform
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

# Create a more sophisticated CNN model
class LandsatCNNImproved(torch.nn.Module):
    def __init__(self, in_channels, num_classes=17):
        super(LandsatCNNImproved, self).__init__()
        
        self.conv1 = torch.nn.Sequential(
            torch.nn.Conv2d(in_channels, 32, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.Conv2d(32, 32, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(32),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(64),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv3 = torch.nn.Sequential(
            torch.nn.Conv2d(64, 128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 128, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(128),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        self.conv4 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.BatchNorm2d(256),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # Calculate output size for a 64x64 input patch
        # After 4 max pooling layers: 64 -> 32 -> 16 -> 8 -> 4
        self.avgpool = torch.nn.AdaptiveAvgPool2d((1, 1))
        
        self.classifier = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(256, 512),
            torch.nn.ReLU(),
            torch.nn.Dropout(0.5),
            torch.nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.avgpool(x)
        x = self.classifier(x)
        return x

def train_and_evaluate(X, y, patch_size=64, num_epochs=50, batch_size=32):
    """Train and evaluate the model using the patch data"""
    # Split data into train, validation, and test sets
    X_tensor = torch.from_numpy(X).float()
    y_tensor = torch.from_numpy(y).long()
    
    dataset = LandsatLCZPatchDataset(X, y)
    
    # Calculate splits
    train_size = int(0.7 * len(dataset))
    val_size = int(0.15 * len(dataset))
    test_size = len(dataset) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        dataset, 
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Create model
    model = LandsatCNNImproved(in_channels=X.shape[1], num_classes=17)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    
    # Define loss and optimizer
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)
    
    # Training loop
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    
    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        train_loss /= len(train_loader)
        train_losses.append(train_loss)
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                val_loss += loss.item()
        
        val_loss /= len(val_loader)
        val_losses.append(val_loss)
        
        # Update learning rate
        scheduler.step(val_loss)
        
        # Save best model
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            torch.save(model.state_dict(), "best_landsat_lcz_model.pth")
        
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
    
    # Load best model for evaluation
    model.load_state_dict(torch.load("best_landsat_lcz_model.pth"))
    
    # Evaluate on test set
    model.eval()
    test_correct = 0
    test_total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch_X, batch_y in test_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            outputs = model(batch_X)
            _, predicted = torch.max(outputs, 1)
            
            test_total += batch_y.size(0)
            test_correct += (predicted == batch_y).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(batch_y.cpu().numpy())
    
    test_acc = test_correct / test_total
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    cm = confusion_matrix(all_labels, all_preds)
    sns.heatmap(cm, annot=True)

KeyError: 'SR_B2'

In [47]:
def patches_to_numpy(patches):
    """ Convert Earth Engine patches to numpy arrays """
    patch_list = []
    labels = []
    for patch in patches.getInfo():
        patch_list.append(np.array(patch['properties']['bands']))
        labels.append(patch['properties']['LCZ'])
    return np.array(patch_list), np.array(labels)

X_train, y_train = patches_to_numpy(training_patches)

EEException: Error in map(ID=0):
GeometryConstructors.Point, argument 'coordinates': Invalid type.
Expected type: List<Number>.
Actual type: Feature.